In [1]:
# Sel 1: Tulis kode aplikasi Streamlit ke file app.py
app_code = """
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk
from nltk.tokenize import word_tokenize

# Unduh data tokenizer NLTK
nltk.download('punkt')

# Fungsi untuk memuat model dan tokenizer
@st.cache_resource
def muat_model_dan_tokenizer(lokasi_model):
    try:
        tokenizer = AutoTokenizer.from_pretrained(lokasi_model, add_prefix_space=True)
        model = AutoModelForTokenClassification.from_pretrained(lokasi_model)
        return model, tokenizer
    except Exception as e:
        st.error(f"Gagal memuat model atau tokenizer: {e}")
        return None, None

# Fungsi untuk memproses teks dan mengekstrak entitas
def prediksi_entitas(teks, model, tokenizer, daftar_label):
    token = word_tokenize(teks)
    encoding = tokenizer(
        token,
        is_split_into_words=True,
        return_offsets_mapping=True,
        truncation=True,
        max_length=512,
        padding='max_length',
        return_tensors='pt'
    )
    model.eval()
    with torch.no_grad():
        input_ids = encoding['input_ids'].to(model.device)
        attention_mask = encoding['attention_mask'].to(model.device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=2).cpu().numpy()[0]
    
    word_ids = encoding.word_ids()
    label_terselaraskan = []
    idx_kata_sebelumnya = None
    for i, idx_kata in enumerate(word_ids):
        if idx_kata is None or idx_kata == idx_kata_sebelumnya:
            continue
        label_terselaraskan.append(daftar_label[predictions[i]])
        idx_kata_sebelumnya = idx_kata
    
    entitas = {'Organisasi': [], 'Angka Finansial': [], 'Tokoh Eksekutif': []}
    entitas_saat_ini = []
    label_saat_ini = None
    
    for token, label in zip(token[:len(label_terselaraskan)], label_terselaraskan):
        if label.startswith('B-'):
            if entitas_saat_ini:
                if label_saat_ini == 'ORG':
                    entitas['Organisasi'].append(' '.join(entitas_saat_ini))
                elif label_saat_ini == 'FIN':
                    entitas['Angka Finansial'].append(' '.join(entitas_saat_ini))
                elif label_saat_ini == 'PER':
                    entitas['Tokoh Eksekutif'].append(' '.join(entitas_saat_ini))
            entitas_saat_ini = [token]
            label_saat_ini = label.split('-')[1]
        elif label.startswith('I-') and label_saat_ini == label.split('-')[1]:
            entitas_saat_ini.append(token)
        else:
            if entitas_saat_ini:
                if label_saat_ini == 'ORG':
                    entitas['Organisasi'].append(' '.join(entitas_saat_ini))
                elif label_saat_ini == 'FIN':
                    entitas['Angka Finansial'].append(' '.join(entitas_saat_ini))
                elif label_saat_ini == 'PER':
                    entitas['Tokoh Eksekutif'].append(' '.join(entitas_saat_ini))
            entitas_saat_ini = []
            label_saat_ini = None
    
    if entitas_saat_ini:
        if label_saat_ini == 'ORG':
            entitas['Organisasi'].append(' '.join(entitas_saat_ini))
        elif label_saat_ini == 'FIN':
            entitas['Angka Finansial'].append(' '.join(entitas_saat_ini))
        elif label_saat_ini == 'PER':
            entitas['Tokoh Eksekutif'].append(' '.join(entitas_saat_ini))
    
    return entitas

# Aplikasi Utama Streamlit
def utama():
    st.title("Ekstraksi Entitas Bisnis dari Artikel Keuangan")
    st.markdown("Masukkan artikel keuangan berbahasa Indonesia untuk mengekstrak entitas seperti nama perusahaan, angka finansial, dan tokoh eksekutif.")

    st.header("Ekstraksi Entitas")
    artikel = st.text_area("Masukkan Artikel Keuangan:", height=200)
    
    lokasi_model = "model/model_indobert"  # Ganti dengan path lokal Anda
    daftar_label = ['O', 'B-FIN', 'I-FIN', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER']
    model, tokenizer = muat_model_dan_tokenizer(lokasi_model)
    
    if st.button("Proses Artikel"):
        if artikel.strip() == "":
            st.warning("Silakan masukkan artikel terlebih dahulu!")
        elif model is None or tokenizer is None:
            st.error("Model atau tokenizer gagal dimuat. Periksa lokasi model!")
        else:
            with st.spinner("Memproses artikel..."):
                entitas = prediksi_entitas(artikel, model, tokenizer, daftar_label)
            
            st.subheader("Hasil Ekstraksi Entitas")
            for jenis_entitas, daftar_entitas in entitas.items():
                st.write(f"**{jenis_entitas}:**")
                if daftar_entitas:
                    for entitas_item in daftar_entitas:
                        st.write(f"- {entitas_item}")
                else:
                    st.write("Tidak ada entitas ditemukan.")
    
    if artikel.strip() != "":
        st.subheader("Teks dengan Entitas Dihighlight")
        token = word_tokenize(artikel)
        entitas = prediksi_entitas(artikel, model, tokenizer, daftar_label)
        teks_dihighlight = ""
        for token_item in token:
            adalah_entitas = False
            for jenis_entitas, daftar_entitas in entitas.items():
                for entitas_item in daftar_entitas:
                    if token_item in entitas_item.split():
                        adalah_entitas = True
                        teks_dihighlight += f"**{token_item}** "
                        break
                if adalah_entitas:
                    break
            if not adalah_entitas:
                teks_dihighlight += f"{token_item} "
        st.markdown(teks_dihighlight)

if __name__ == "__main__":
    utama()
"""

with open("app.py", "w") as f:
    f.write(app_code)
print("File app.py telah dibuat di direktori kerja Anda.")

File app.py telah dibuat di direktori kerja Anda.


In [2]:
!streamlit run app.py

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\ASUS\AppData\Local\Programs\Python\Python311\Scripts\streamlit.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'streamlit'


In [1]:
pip install nltk

In [3]:
pip install streamlit

  Using cached streamlit-1.45.1-py3-none-any.whl.metadata (8.9 kB)
Using cached streamlit-1.45.1-py3-none-any.whl (9.9 MB)
Note: you may need to restart the kernel to use updated packages.
